# Práctica 5 Automatización y Optimización Avanzada

> Objetivo central: **"Automatiza un stack CRUD completo (modelo, API, tests e infraestructura) orquestando LLMs con LangChain de forma reproducible y medible."**

## ¿Qué voy a lograr y por qué importa?
En esta práctica construyes un **pipeline automatizado** que, partiendo de una configuración declarativa (YAML / Pydantic), genera:
- Modelos Pydantic validados
- Router FastAPI empresarial (CRUD + paginación + auth opcional)
- Suite de tests Pytest
- Infraestructura (Dockerfile, migración Alembic)
- Métricas de eficiencia de la generación

Esto refleja un caso real: equipos que necesitan **acelerar el scaffolding backend** manteniendo estándares de calidad. Aprenderás a usar **LangChain Expression Language (LCEL)** y `RunnableParallel` para ejecutar tareas en paralelo y encadenar dependencias.

Problemas reales que esto resuelve:
- Onboarding lento: crear cada CRUD manualmente tarda horas.
- Inconsistencia entre servicios: estilos diferentes de validación / logs.
- Falta de medición: se generan cosas con IA pero sin métricas.
- Riesgo técnico: prompts desordenados, sin control de dependencias.

Solución mostrada: un **pipeline determinista** donde cada bloque tiene una responsabilidad clara. Así escalas generación de servicios sin sacrificar mantenibilidad.

Rol de las piezas:
- `ResourceConfig` y `FieldConfig`: contrato declarativo de tu recurso.
- Prompts especializados (model, router, tests, infra): separación de dominios (Domain Prompting).
- `RunnableParallel`: acelera la generación base (modelo + config) y luego deriva dependientes.
- `Structured Output`: fuerza esquemas (`InfrastructureComponents`).
- Métricas: base para gobernanza y ROI de IA.

| Concepto | Idea-faro | Analogía |
|----------|-----------|----------|
| LCEL | Orquesta modular | "LEGO de flujos LLM" |
| `RunnableParallel` | Paralelismo declarativo | "Carriles simultáneos" |
| Config → Artefactos | Infra como código pero para scaffolding | "Terraform de tu backend" |
| Prompts especializados | Principio de responsabilidad única | "Microservicios cognitivos" |
| Métricas | Observabilidad del pipeline | "Tablero de control DevOps" |
| Structured Output | Control sintáctico | "Molde para la arcilla del modelo" |


## Práctica paso a paso

### Parte 1: Setup del Entorno

Configuraremos un entorno completo con herramientas de análisis y generación automatizada.

In [2]:
!pip install langchain langchain-openai langchain-community fastapi uvicorn pydantic pytest httpx python-dotenv jinja2 pyyaml click rich

In [3]:
import os
import yaml
import json
import ast
import time
import logging
from pathlib import Path
from typing import Dict, List, Optional, Any, Literal
from dataclasses import dataclass
from datetime import datetime
from jinja2 import Template

# LangChain imports
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser, StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnableLambda
from langchain_core.callbacks import BaseCallbackHandler
from pydantic import BaseModel, Field


In [4]:

from dotenv import load_dotenv
load_dotenv()

if not os.getenv("OPENAI_API_KEY"):
    print("⚠️  Configura OPENAI_API_KEY en tu archivo .env")
else:
    print("✅ OpenAI API Key configurada")

# Configurar modelo
model = ChatOpenAI(model="gpt-5", temperature=0)
print("🤖 Modelo listo")

✅ OpenAI API Key configurada
🤖 Modelo listo


### 4.2 Modelos de Configuración
Implementaremos un sistema avanzado de generación CRUD usando RunnableParallel y configuración YAML.

In [5]:
from typing import Literal, Union

# Modelos de configuración

class FieldConfig(BaseModel):
    name: str
    type: Literal["int", "float", "str", "bool"]
    description: str = ""
    constraints: Optional[Dict[str, Any]] = None

class ResourceConfig(BaseModel):
    resource_name: str
    class_name: str
    fields: List[FieldConfig]
    auth_required: bool = False
    cache_enabled: bool = False
    pagination: bool = True
    soft_delete: bool = False

In [6]:
# 2. Crear configuración de ejemplo
sample_config = ResourceConfig(
    resource_name="product",
    class_name="Product", 
    fields=[
        FieldConfig(name="name", type="str", description="Nombre del producto", 
                   constraints={"min_length": 1, "max_length": 100}),
        FieldConfig(name="price", type="float", description="Precio en USD",
                   constraints={"ge": 0, "le": 999999}),
        FieldConfig(name="category", type="str", required=False, description="Categoría"),
        FieldConfig(name="stock", type="int", description="Cantidad en inventario",
                   constraints={"ge": 0})
    ],
    auth_required=True,
    cache_enabled=True,
    pagination=True,
    soft_delete=True
)

# Guardar configuración para uso posterior
config_path = Path("resource_configs") 
config_path.mkdir(exist_ok=True)
with open(config_path / "product_config.yaml", "w") as f:
    yaml.dump(sample_config.dict(), f, default_flow_style=False)



/tmp/ipykernel_3788136/77762200.py:24: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  yaml.dump(sample_config.dict(), f, default_flow_style=False)


### Parte 3: Generadores Especializados con RunnableParallel

Crearemos generadores especializados que trabajen en paralelo para máxima eficiencia.

Se crean 4 generadores:
1. `model_generator`: produce modelos Pydantic (entrada, salida, update, validaciones y validators).
2. `router_generator`: crea router FastAPI con CRUD completo y middlewares condicionales.
3. `tests_generator`: diseña suite Pytest (unit, integration, performance básico, edge cases).
4. `infra_generator`: con `with_structured_output` para garantizar campos (`dockerfile`, `migration`, etc.).

Diseño de prompts: cada uno declara explícitamente criterios de calidad (ej. "nivel PRODUCCIÓN", "validaciones específicas", "logging estructurado"). Esto reduce alucinaciones.


In [7]:
import json
from langchain_core.output_parsers import StrOutputParser

# GENERADORES ESPECIALIZADOS 

class GeneratedComponents(BaseModel):
    modelo_pydantic: str = Field(description="Modelo Pydantic con validaciones avanzadas")
    router_fastapi: str = Field(description="Router FastAPI con CRUD completo")
    tests_pytest: str = Field(description="Suite de tests exhaustiva")
    alembic_migration: str = Field(description="Migración Alembic para base de datos")
    dockerfile: str = Field(description="Dockerfile optimizado")


# Templates
model_generator =(ChatPromptTemplate.from_messages([
        ("system", """Eres experto en Pydantic y diseño de APIs con FastAPI.
        Genera modelos Pydantic de nivel PRODUCCIÓN con:
        - Validaciones específicas por tipo de campo
        - Docstrings detallados con ejemplos
        - Field constraints apropiados
        - Validators personalizados para lógica de negocio
        - Modelos de entrada, salida y actualización separados
        """),
                ("human", """
        CONFIGURACIÓN:
        Resource: {resource_name}
        Class: {class_name}
        Fields: {fields}
        Features: auth={auth_required}, cache={cache_enabled}, soft_delete={soft_delete}

        Genera modelos Pydantic profesionales con validaciones robustas.
        """)
    ])
    | model
    | StrOutputParser()
)


In [8]:
model_generator

ChatPromptTemplate(input_variables=['auth_required', 'cache_enabled', 'class_name', 'fields', 'resource_name', 'soft_delete'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Eres experto en Pydantic y diseño de APIs con FastAPI.\n        Genera modelos Pydantic de nivel PRODUCCIÓN con:\n        - Validaciones específicas por tipo de campo\n        - Docstrings detallados con ejemplos\n        - Field constraints apropiados\n        - Validators personalizados para lógica de negocio\n        - Modelos de entrada, salida y actualización separados\n        '), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['auth_required', 'cache_enabled', 'class_name', 'fields', 'resource_name', 'soft_delete'], input_types={}, partial_variables={}, template='\n        CONFIGURACIÓN:\n        Resource: {resource_name}\n        Class: {class_name}\

In [9]:
# 2. Generador de routers FastAPI completos
router_generator = (
    ChatPromptTemplate.from_messages([
        ("system", """Eres arquitecto senior FastAPI especialista en APIs RESTful.
        Genera routers de PRODUCCIÓN con:
        - Endpoints CRUD completos (GET, POST, PUT, PATCH, DELETE)
        - Paginación, filtrado y ordenamiento
        - Manejo de errores HTTP consistente
        - Documentación OpenAPI rica
        - Middlewares de auth y cache según configuración
        - Logging estructurado
        - Validación de permisos
        """),
                ("human", """
        MODELO PYDANTIC:
        {modelo_pydantic}

        CONFIGURACIÓN:
        {config}

        Genera router FastAPI de nivel empresarial.
    """)
    ])
    | model
    | StrOutputParser()
)

In [13]:
router_generator

ChatPromptTemplate(input_variables=['config', 'modelo_pydantic'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Eres arquitecto senior FastAPI especialista en APIs RESTful.\n        Genera routers de PRODUCCIÓN con:\n        - Endpoints CRUD completos (GET, POST, PUT, PATCH, DELETE)\n        - Paginación, filtrado y ordenamiento\n        - Manejo de errores HTTP consistente\n        - Documentación OpenAPI rica\n        - Middlewares de auth y cache según configuración\n        - Logging estructurado\n        - Validación de permisos\n        '), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['config', 'modelo_pydantic'], input_types={}, partial_variables={}, template='\n        MODELO PYDANTIC:\n        {modelo_pydantic}\n\n        CONFIGURACIÓN:\n        {config}\n\n        Genera router FastAPI de nivel empresarial.\n    ')

In [14]:
# 3. Generador de tests exhaustivos
tests_generator = (
    ChatPromptTemplate.from_messages([
        ("system", """Eres QA Lead especialista en testing de APIs.
        Genera suite de tests COMPREHENSIVA con:
        - Tests unitarios para cada endpoint
        - Tests de integración end-to-end
        - Tests de performance básicos
        - Tests de seguridad (auth, validation)
        - Tests de casos borde y error handling
        - Fixtures y mocks apropiados
        - Cobertura de al menos 90%
        """),
                ("human", """
        ROUTER FASTAPI:
        {router_fastapi}

        MODELO PYDANTIC:
        {modelo_pydantic}

        CONFIGURACIÓN:
        {config}

        Genera tests pytest de nivel empresarial.
        """)
    ])
    | model
    | StrOutputParser()
)

In [15]:
tests_generator

ChatPromptTemplate(input_variables=['config', 'modelo_pydantic', 'router_fastapi'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Eres QA Lead especialista en testing de APIs.\n        Genera suite de tests COMPREHENSIVA con:\n        - Tests unitarios para cada endpoint\n        - Tests de integración end-to-end\n        - Tests de performance básicos\n        - Tests de seguridad (auth, validation)\n        - Tests de casos borde y error handling\n        - Fixtures y mocks apropiados\n        - Cobertura de al menos 90%\n        '), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['config', 'modelo_pydantic', 'router_fastapi'], input_types={}, partial_variables={}, template='\n        ROUTER FASTAPI:\n        {router_fastapi}\n\n        MODELO PYDANTIC:\n        {modelo_pydantic}\n\n        CONFIGURACIÓN:\n        {config}\n\

In [31]:
# 4. Modelo para infraestructura
class InfrastructureComponents(BaseModel):
    dockerfile: str = Field(description="Dockerfile multi-stage optimizado")
    migration: str = Field(description="Migración Alembic con índices")
    docker_compose: str = Field(description="Docker-compose para desarrollo", default="")
    deployment_script: str = Field(description="Script de deployment", default="")

# 4. Generador de infraestructura (Docker, migrations) - corregido
infra_generator = ChatPromptTemplate.from_messages([
    ("system", """Eres DevOps senior especialista en containerización y databases.
    Genera infraestructura de PRODUCCIÓN:
    - Dockerfile multi-stage optimizado
    - Migración Alembic con índices apropiados
    - Docker-compose para desarrollo
    - Scripts de deployment
    """),
        ("human", """
    CONFIGURACIÓN:
    {config}

    MODELO PYDANTIC:
    {modelo_pydantic}

    Genera infraestructura completa para producción.
    """)
]) | model.with_structured_output(InfrastructureComponents)

### 4.4 Orquestación LCEL
Función `create_advanced_crud_pipeline()`:
- Fase base paralela: genera `modelo` y pasa `config` intacta.
- Lambda intermedia `_generate_dependent_components`: usa salida anterior para generar router y luego en paralelo tests + infraestructura.
- Se empaqueta todo en un `GeneratedComponents` final.

Ventaja: minimiza latencia (paraleliza lo que no depende) y mantiene orden lógico de dependencias (modelo → router → tests/infra).

In [25]:
def _generate_dependent_components(base_components):
    config = base_components["config"]
    modelo = base_components["modelo"]

    router = router_generator.invoke({
        "modelo_pydantic": modelo,
        "config": json.dumps(config, indent=2, ensure_ascii=False)
    })

    parallel_final = RunnableParallel({
        "tests": tests_generator,
        "infra": infra_generator
    })

    final_components = parallel_final.invoke({
        "router_fastapi": router,
        "modelo_pydantic": modelo,
        "config": json.dumps(config, indent=2, ensure_ascii=False)
    })

    infra = final_components["infra"]
    return GeneratedComponents(
        modelo_pydantic=modelo,
        router_fastapi=router,
        tests_pytest=final_components["tests"],
        alembic_migration=infra.migration if infra else "",
        dockerfile=infra.dockerfile if infra else ""
    )

def create_advanced_crud_pipeline():
    parallel_base = RunnableParallel({
        "modelo": model_generator,
        "config": RunnableLambda(lambda x: x)
    })
    return parallel_base | RunnableLambda(_generate_dependent_components)


In [26]:
crud_pipeline = create_advanced_crud_pipeline()

crud_pipeline

{
  modelo: ChatPromptTemplate(input_variables=['auth_required', 'cache_enabled', 'class_name', 'fields', 'resource_name', 'soft_delete'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Eres experto en Pydantic y diseño de APIs con FastAPI.\n        Genera modelos Pydantic de nivel PRODUCCIÓN con:\n        - Validaciones específicas por tipo de campo\n        - Docstrings detallados con ejemplos\n        - Field constraints apropiados\n        - Validators personalizados para lógica de negocio\n        - Modelos de entrada, salida y actualización separados\n        '), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['auth_required', 'cache_enabled', 'class_name', 'fields', 'resource_name', 'soft_delete'], input_types={}, partial_variables={}, template='\n        CONFIGURACIÓN:\n        Resource: {resource_name}\n        Class: {

### 4.5 Ejecución + Métricas
Se prepara input a partir de `sample_config` y se invoca `crud_pipeline.invoke(pipeline_input)`.

Métricas recolectadas manualmente (se esboza callback pero no se conecta en la ejecución actual):
- Tiempo total de generación
- Conteo de líneas por componente
- Número de clases, endpoints, validators, fixtures, asserts
- Cálculo de ROI: (tiempo manual estimado / tiempo IA)


In [27]:
# EJECUCIÓN DEL PIPELINE CON MÉTRICAS AVANZADAS

# 1. Callback para capturar métricas detalladas
class CRUDGenerationMetrics(BaseCallbackHandler):
    def __init__(self):
        self.metrics = {
            "start_time": None,
            "end_time": None,
            "components_generated": 0,
            "total_tokens": 0,
            "llm_calls": 0,
            "parallel_executions": 0,
            "errors": 0,
            "component_times": {}
        }
        self.current_component = None
        self.component_start = None
    
    def on_chain_start(self, serialized, inputs, **kwargs):
        if not self.metrics["start_time"]:
            self.metrics["start_time"] = time.time()
        self.component_start = time.time()
    
    def on_chain_end(self, outputs, **kwargs):
        if self.component_start:
            duration = time.time() - self.component_start
            component_name = self.current_component or "unknown"
            self.metrics["component_times"][component_name] = duration
            self.metrics["components_generated"] += 1
        
        self.metrics["end_time"] = time.time()
    
    def on_llm_start(self, serialized, prompts, **kwargs):
        self.metrics["llm_calls"] += 1
    
    def on_llm_end(self, response, **kwargs):
        if hasattr(response, 'llm_output') and response.llm_output:
            token_usage = response.llm_output.get('token_usage', {})
            self.metrics["total_tokens"] += token_usage.get('total_tokens', 0)
    
    def on_chain_error(self, error, **kwargs):
        self.metrics["errors"] += 1
    
    def get_summary(self):
        total_time = (self.metrics["end_time"] or time.time()) - (self.metrics["start_time"] or time.time())
        return {
            **self.metrics,
            "total_time": total_time,
            "avg_time_per_component": total_time / max(self.metrics["components_generated"], 1),
            "tokens_per_second": self.metrics["total_tokens"] / max(total_time, 1),
            "success_rate": (self.metrics["components_generated"] - self.metrics["errors"]) / max(self.metrics["components_generated"], 1) * 100
        }

# 2. Ejecutar generación con métricas
print("🚀 INICIANDO GENERACIÓN CRUD AUTOMATIZADA...")
print("=" * 60)

metrics_callback = CRUDGenerationMetrics()

🚀 INICIANDO GENERACIÓN CRUD AUTOMATIZADA...


In [21]:
sample_config.resource_name

'product'

In [29]:
# Preparar input para el pipeline
pipeline_input = {
    "resource_name": sample_config.resource_name,
    "class_name": sample_config.class_name,
    "fields": [f.dict() for f in sample_config.fields],
    "auth_required": sample_config.auth_required,
    "cache_enabled": sample_config.cache_enabled,
    "soft_delete": sample_config.soft_delete
}

pipeline_input

/tmp/ipykernel_3788136/4071791889.py:5: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  "fields": [f.dict() for f in sample_config.fields],


{'resource_name': 'product',
 'class_name': 'Product',
 'fields': [{'name': 'name',
   'type': 'str',
   'description': 'Nombre del producto',
   'constraints': {'min_length': 1, 'max_length': 100}},
  {'name': 'price',
   'type': 'float',
   'description': 'Precio en USD',
   'constraints': {'ge': 0, 'le': 999999}},
  {'name': 'category',
   'type': 'str',
   'description': 'Categoría',
   'constraints': None},
  {'name': 'stock',
   'type': 'int',
   'description': 'Cantidad en inventario',
   'constraints': {'ge': 0}}],
 'auth_required': True,
 'cache_enabled': True,
 'soft_delete': True}

In [32]:
# Ejecutar el pipeline
start_generation = time.time()
generated_components = crud_pipeline.invoke(pipeline_input)
generation_time = time.time() - start_generation

print(f"Generación Completa en {generation_time}")

Generación Completa en 446.1253263950348


In [35]:
generated_components

GeneratedComponents(modelo_pydantic='from __future__ import annotations\n\nfrom datetime import datetime, timezone\nfrom decimal import Decimal, ROUND_HALF_UP, InvalidOperation\nfrom typing import Annotated, Optional\nfrom uuid import UUID\n\nfrom pydantic import (\n    BaseModel,\n    ConfigDict,\n    Field,\n    StringConstraints,\n    StrictInt,\n    field_validator,\n    model_validator,\n)\n\n\nNameStr = Annotated[\n    str,\n    StringConstraints(\n        min_length=1,\n        max_length=100,\n        strip_whitespace=True,\n    ),\n]\n\nCategoryStr = Annotated[\n    str,\n    StringConstraints(\n        strip_whitespace=True,\n    ),\n]\n\n\nclass ProductBase(BaseModel):\n    """\n    Modelo base del recurso Product.\n\n    Incluye validaciones y normalización:\n    - name: recorta y colapsa espacios múltiples, no permite caracteres de control.\n    - price: convierte a Decimal, redondea a 2 decimales (ROUND_HALF_UP), y valida rango.\n    - category: recorta y colapsa espacios

In [36]:
# 3. Análisis de los componentes generados
print("\n" + "=" * 60)
print("📊 ANÁLISIS DE COMPONENTES GENERADOS")
print("=" * 60)

components_analysis = {
    "modelo": {
        "lines": generated_components.modelo_pydantic.count('\n'),
        "classes": generated_components.modelo_pydantic.count('class '),
        "validations": generated_components.modelo_pydantic.count('Field('),
        "validators": generated_components.modelo_pydantic.count('@validator')
    },
    "router": {
        "lines": generated_components.router_fastapi.count('\n'),
        "endpoints": len([x for x in ['@app.get', '@app.post', '@app.put', '@app.delete'] 
                        if x in generated_components.router_fastapi]),
        "error_handling": generated_components.router_fastapi.count('HTTPException'),
        "documentation": generated_components.router_fastapi.count('summary=')
    },
    "tests": {
        "lines": generated_components.tests_pytest.count('\n'),
        "test_functions": generated_components.tests_pytest.count('def test_'),
        "assertions": generated_components.tests_pytest.count('assert '),
        "fixtures": generated_components.tests_pytest.count('@pytest.fixture')
    }
}

for component, analysis in components_analysis.items():
    print(f"\n🔍 {component.upper()}:")
    for metric, value in analysis.items():
        print(f"  • {metric.replace('_', ' ').title()}: {value}")


📊 ANÁLISIS DE COMPONENTES GENERADOS

🔍 MODELO:
  • Lines: 387
  • Classes: 4
  • Validations: 19
  • Validators: 0

🔍 ROUTER:
  • Lines: 838
  • Endpoints: 0
  • Error Handling: 5
  • Documentation: 8

🔍 TESTS:
  • Lines: 674
  • Test Functions: 15
  • Assertions: 78
  • Fixtures: 8


In [37]:
print(generated_components.router_fastapi)

A continuación tienes un router FastAPI de nivel empresarial para el recurso Product, alineado con tus modelos Pydantic y la configuración (auth_required=true, cache_enabled=true, soft_delete=true). Incluye:

- Endpoints CRUD completos (GET list, GET by id, POST, PUT, PATCH, DELETE soft), y extras restore y hard delete.
- Paginación, filtrado y ordenamiento.
- ETag/If-None-Match/If-Match (caching y concurrencia optimista).
- Manejo de errores consistente con payload estructurado.
- Validación de permisos y autenticación vía dependencias.
- Logging estructurado.
- Documentación OpenAPI rica y ejemplos.

Copia/pega en tu proyecto y reemplaza la implementación del repositorio por tu capa de datos (SQLAlchemy, etc.).


from __future__ import annotations

import hashlib
import logging
from datetime import datetime, timezone
from typing import Annotated, Iterable, Optional
from uuid import UUID, uuid4

from fastapi import (
    APIRouter,
    Depends,
    HTTPException,
    Path,
    Query,


### 4.6 Persistencia de Artefactos
Escribe a carpeta `generated_product_api/` solo archivos con contenido.
- `models.py`
- `routes.py`
- `test_api.py`
- `Dockerfile`
- `migration.py`


In [38]:
# 4. Guardar componentes generados
output_dir = Path(f"generated_{sample_config.resource_name}_api")
output_dir.mkdir(exist_ok=True)

files_created = {
    "models.py": generated_components.modelo_pydantic,
    "routes.py": generated_components.router_fastapi,
    "test_api.py": generated_components.tests_pytest,
    "Dockerfile": generated_components.dockerfile,
    "migration.py": generated_components.alembic_migration
}

for filename, content in files_created.items():
    if content.strip():  # Solo crear archivos con contenido
        file_path = output_dir / filename
        file_path.write_text(content)

print(f"\n💾 Archivos generados en: {output_dir}")
print(f"📁 Total archivos: {len([f for f in files_created.values() if f.strip()])}")


💾 Archivos generados en: generated_product_api
📁 Total archivos: 5


Si venías de:
- Prompt Engineering básico → ahora formalizas prompts como componentes reutilizables.
- FastAPI manual → ahora generas scaffolding consistente.
- DevOps / Infra → introduces IaC-like patterns para artefactos de backend.

Idea principal para recordar: **"Estandariza y paraleliza lo generable; reserva tu tiempo humano para lo verdaderamente diferencial."**

Checklist mental (READY):
- ¿Tengo config declarativa? ✅
- ¿Prompts con criterios explícitos? ✅
- ¿Control de dependencias y orden? ✅
- ¿Métricas de eficiencia? ✅
- ¿Artefactos persistidos y reutilizables? ✅

Si venías de:
- Prompt Engineering básico → ahora formalizas prompts como componentes reutilizables.
- FastAPI manual → ahora generas scaffolding consistente.
- DevOps / Infra → introduces IaC-like patterns para artefactos de backend.

Idea principal para recordar: **"Estandariza y paraleliza lo generable; reserva tu tiempo humano para lo verdaderamente diferencial."**

Checklist mental (READY):
- ¿Tengo config declarativa? ✅
- ¿Prompts con criterios explícitos? ✅
- ¿Control de dependencias y orden? ✅
- ¿Métricas de eficiencia? ✅
- ¿Artefactos persistidos y reutilizables? ✅

## Parte 1: Pipeline de Refactoring Inteligente

Implementaremos un sistema que analiza código existente y propone mejoras automáticas basadas en métricas.

In [43]:
# LangChain imports
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser, StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnableLambda
from langchain_core.callbacks import BaseCallbackHandler
from pydantic import BaseModel, Field

# PIPELINE DE REFACTORING INTELIGENTE
from langchain_core.runnables import RunnableParallel, RunnableLambda
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List, Dict, Any

In [45]:
class RefactoringAnalysis(BaseModel):
    """Análisis integral para refactoring"""
    code_issues: List[str] = Field(description="Lista de problemas detectados en el código")
    refactoring_priority: str = Field(description="Alta/Media/Baja prioridad de refactoring")
    improvement_suggestions: List[str] = Field(description="Sugerencias específicas de mejora")
    estimated_effort: str = Field(description="Estimación de esfuerzo: Bajo/Medio/Alto")
    maintainability_score: int = Field(description="Puntuación de mantenibilidad (1-10)")

class RefactoredCode(BaseModel):
    """Código refactorizado con mejoras"""
    improved_code: str = Field(description="Código mejorado y refactorizado")
    changes_summary: List[str] = Field(description="Resumen de cambios realizados")
    performance_improvements: List[str] = Field(description="Mejoras de rendimiento aplicadas")
    code_style_fixes: List[str] = Field(description="Correcciones de estilo de código")

# Parsers
analysis_parser = PydanticOutputParser(pydantic_object=RefactoringAnalysis)
refactoring_parser = PydanticOutputParser(pydantic_object=RefactoredCode)

In [47]:
# Pipeline de análisis y refactoring
def create_refactoring_pipeline():
    """Crear pipeline inteligente de refactoring"""
    
    # Análisis técnico
    technical_analysis = RunnableLambda(
        lambda x: f"""
        Analiza el siguiente código y proporciona un análisis técnico detallado:

        CÓDIGO:
        {x['code']}

        MÉTRICAS DETECTADAS:
        {x['metrics']}

        PATRONES DETECTADOS:
        {x['patterns']}

        Proporciona un análisis integral considerando:
        1. Problemas de complejidad y mantenibilidad
        2. Anti-patrones detectados
        3. Oportunidades de mejora
        4. Priorización de refactoring

        {analysis_parser.get_format_instructions()}
        """
    ) | model | analysis_parser

    # Refactoring inteligente
    intelligent_refactoring = RunnableLambda(
        lambda x: f"""
        Refactoriza el siguiente código basándote en el análisis técnico:

        CÓDIGO ORIGINAL:
        {x['code']}

        ANÁLISIS TÉCNICO:
        {x['analysis']}

        DIRECTRICES DE REFACTORING:
        1. Reducir complejidad ciclomática manteniendo funcionalidad
        2. Aplicar principios SOLID y clean code
        3. Mejorar legibilidad y mantenibilidad
        4. Optimizar rendimiento donde sea posible
        5. Mantener compatibilidad con APIs existentes
        6. Agregar documentación y tipado donde falte

        IMPORTANTE: El código debe seguir mejores prácticas de FastAPI y Pydantic.

        {refactoring_parser.get_format_instructions()}
        """
    ) | model | refactoring_parser

    # Pipeline paralelo
    refactoring_pipeline = RunnableParallel({
        "analysis": technical_analysis,
        "code": RunnableLambda(lambda x: x["code"]),
        "metrics": RunnableLambda(lambda x: x["metrics"]),
        "patterns": RunnableLambda(lambda x: x["patterns"])
    }) | RunnableLambda(
        lambda x: {
            "code": x["code"],
            "analysis": x["analysis"],
            "metrics": x["metrics"],
            "patterns": x["patterns"]
        }
    ) | RunnableParallel({
        "original": RunnableLambda(lambda x: x),
        "refactored": intelligent_refactoring
    })

    return refactoring_pipeline

# Crear pipeline
refactoring_system = create_refactoring_pipeline()

print("🚀 Pipeline de refactoring inteligente creado")
print("  • Análisis técnico automático")
print("  • Refactoring basado en métricas")
print("  • Optimización de código paralela")

🚀 Pipeline de refactoring inteligente creado
  • Análisis técnico automático
  • Refactoring basado en métricas
  • Optimización de código paralela


In [48]:
refactoring_system

{
  analysis: RunnableLambda(lambda x: f"\nAnaliza el siguiente código y proporciona un análisis técnico detallado:\n\nCÓDIGO:\n{x['code']}\n\nMÉTRICAS DETECTADAS:\n{x['metrics']}\n\nPATRONES DETECTADOS:\n{x['patterns']}\n\nProporciona un análisis integral considerando:\n1. Problemas de complejidad y mantenibilidad\n2. Anti-patrones detectados\n3. Oportunidades de mejora\n4. Priorización de refactoring\n\n{analysis_parser.get_format_instructions()}\n")
            | ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x7f1cf5d384a0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7f1e939c3bf0>, root_client=<openai.OpenAI object at 0x7f1cf5e21760>, root_async_client=<openai.AsyncOpenAI object at 0x7f1cf5ce0b00>, model_name='gpt-5', model_kwargs={}, openai_api_key=SecretStr('**********'))
            | PydanticOutputParser(pydantic_object=<class '__main__.RefactoringAnalysis'>),
  code: RunnableLambda(...),
  me

In [49]:
# DEMOSTRACIÓN: Refactoring inteligente de código legacy
import time

# Código de ejemplo con problemas detectables
sample_legacy_code = '''
def process_user_data(data):
    print("Processing user data...")
    result = []
    
    # TODO: Add input validation
    for item in data:
        print(f"Processing item: {item}")
        
        if item["age"] > 18 and item["status"] == "active":
            processed = {}
            processed["id"] = item["id"]
            processed["name"] = item["name"] 
            processed["email"] = item["email"]
            processed["age"] = item["age"]
            processed["processed_at"] = time.time()
            
            # Complex business logic
            if item["subscription"] == "premium":
                if item["region"] == "US":
                    processed["discount"] = 0.1
                    if item["loyalty_years"] > 5:
                        processed["discount"] = 0.15
                        if item["referrals"] > 10:
                            processed["discount"] = 0.2
                elif item["region"] == "EU":
                    processed["discount"] = 0.08
                    if item["loyalty_years"] > 3:
                        processed["discount"] = 0.12
                else:
                    processed["discount"] = 0.05
            else:
                processed["discount"] = 0.0
            
            result.append(processed)
            print(f"Processed user: {processed['name']}")
    
    print(f"Total processed: {len(result)}")
    return result
'''

print("📋 Código legacy de ejemplo cargado")
print("Problemas visibles:")
print("  • Función muy larga y compleja")
print("  • Lógica de negocio anidada")
print("  • TODOs sin resolver")
print("  • Uso excesivo de print")
print("  • Falta de validaciones")

📋 Código legacy de ejemplo cargado
Problemas visibles:
  • Función muy larga y compleja
  • Lógica de negocio anidada
  • TODOs sin resolver
  • Uso excesivo de print
  • Falta de validaciones


In [50]:
# FUNCIONES DE ANÁLISIS DE CÓDIGO (faltantes)
import ast
import logging

def analyze_code_complexity(code: str) -> Dict[str, Any]:
    """Analizar complejidad del código usando AST"""
    try:
        tree = ast.parse(code)
        
        complexity_score = 0
        functions = []
        issues = []
        
        # Analizar nodos del AST
        for node in ast.walk(tree):
            # Contar estructuras de control (aumentan complejidad)
            if isinstance(node, (ast.If, ast.For, ast.While, ast.With)):
                complexity_score += 1
            # Contar funciones
            elif isinstance(node, ast.FunctionDef):
                functions.append(node.name)
                # Calcular complejidad de la función
                func_complexity = sum(1 for n in ast.walk(node) 
                                    if isinstance(n, (ast.If, ast.For, ast.While, ast.With)))
                if func_complexity > 5:
                    issues.append(f"Función '{node.name}' muy compleja ({func_complexity} puntos)")
        
        # Métricas básicas
        lines = len([line for line in code.split('\n') if line.strip()])
        code_lines = len([line for line in code.split('\n') 
                         if line.strip() and not line.strip().startswith('#')])
        
        # Calcular índice de mantenibilidad (escala 0-100)
        maintainability_index = max(0, 100 - complexity_score * 2 - (code_lines / 10))
        
        # Detectar problemas adicionales
        if 'TODO' in code:
            issues.append("TODOs pendientes en el código")
        if code.count('print(') > 3:
            issues.append("Uso excesivo de print statements")
        if len(functions) == 0:
            issues.append("No hay funciones definidas")
        
        return {
            "complexity_score": complexity_score,
            "average_complexity": complexity_score / max(len(functions), 1),
            "maintainability_index": maintainability_index,
            "code_lines": code_lines,
            "total_lines": lines,
            "functions_count": len(functions),
            "complex_functions": [f for f in functions if f in [issue.split("'")[1] for issue in issues if "muy compleja" in issue]],
            "issues": issues
        }
        
    except SyntaxError as e:
        return {
            "complexity_score": 100,  # Código con errores de sintaxis = alta complejidad
            "average_complexity": 100,
            "maintainability_index": 0,
            "code_lines": 0,
            "total_lines": len(code.split('\n')),
            "functions_count": 0,
            "complex_functions": [],
            "issues": [f"Error de sintaxis: {str(e)}"]
        }

def analyze_code_patterns(code: str) -> Dict[str, List[str]]:
    """Analizar patrones y anti-patrones en el código"""
    good_patterns = []
    anti_patterns = []
    suggestions = []
    
    # Detectar patrones positivos
    if 'def ' in code:
        good_patterns.append("Funciones definidas (modularidad)")
    if '"""' in code or "'''" in code:
        good_patterns.append("Documentación con docstrings")
    if 'try:' in code and 'except' in code:
        good_patterns.append("Manejo de errores implementado")
    if 'class ' in code:
        good_patterns.append("Uso de clases (OOP)")
    if 'import ' in code:
        good_patterns.append("Imports organizados")
    
    # Detectar anti-patrones
    if code.count('if ') > 5 and 'elif' not in code:
        anti_patterns.append("Múltiples if anidados (considerar elif)")
    if 'print(' in code and code.count('print(') > 2:
        anti_patterns.append("Uso excesivo de print (usar logging)")
    if 'TODO' in code:
        anti_patterns.append("TODOs sin resolver")
    if len([line for line in code.split('\n') if len(line) > 100]) > 0:
        anti_patterns.append("Líneas muy largas (>100 caracteres)")
    
    # Contar niveles de indentación
    max_indent = max([len(line) - len(line.lstrip()) for line in code.split('\n')] + [0])
    if max_indent > 16:  # >4 niveles de indentación
        anti_patterns.append(f"Anidamiento profundo ({max_indent//4} niveles)")
    
    # Generar sugerencias
    if 'TODO' in code:
        suggestions.append("Resolver TODOs pendientes")
    if anti_patterns:
        suggestions.append("Refactorizar para reducir complejidad")
    if 'print(' in code:
        suggestions.append("Reemplazar prints con logging estructurado")
    if max_indent > 12:
        suggestions.append("Extraer funciones para reducir anidamiento")
    if code.count('def ') == 0:
        suggestions.append("Dividir código en funciones reutilizables")
    
    return {
        "good_patterns": good_patterns,
        "anti_patterns": anti_patterns,
        "suggestions": suggestions,
        "complexity_indicators": {
            "max_indentation": max_indent,
            "function_count": code.count('def '),
            "print_statements": code.count('print('),
            "todo_count": code.count('TODO')
        }
    }

print("✅ Funciones de análisis de código agregadas:")

✅ Funciones de análisis de código agregadas:


In [51]:
# ANÁLISIS AUTOMÁTICO DEL CÓDIGO LEGACY
import asyncio

async def analyze_legacy_code():
    """Ejecutar análisis completo del código legacy"""
    print("🔍 Iniciando análisis automático del código legacy...")
    
    # 1. Análisis de complejidad
    complexity_analysis = analyze_code_complexity(sample_legacy_code)
    print(f"\n📊 ANÁLISIS DE COMPLEJIDAD:")
    print(f"  • Complejidad máxima: {complexity_analysis.get('complexity_score', 'N/A')}")
    print(f"  • Complejidad promedio: {complexity_analysis.get('average_complexity', 'N/A'):.1f}")
    print(f"  • Índice mantenibilidad: {complexity_analysis.get('maintainability_index', 'N/A'):.1f}")
    print(f"  • Líneas de código: {complexity_analysis.get('code_lines', 'N/A')}")
    print(f"  • Funciones complejas: {complexity_analysis.get('complex_functions', [])}")
    
    if complexity_analysis.get('issues'):
        print(f"  ⚠️  PROBLEMAS DETECTADOS:")
        for issue in complexity_analysis['issues']:
            print(f"      - {issue}")
    
    # 2. Análisis de patrones
    pattern_analysis = analyze_code_patterns(sample_legacy_code)
    print(f"\n🔍 ANÁLISIS DE PATRONES:")
    
    if pattern_analysis['good_patterns']:
        print(f"  ✅ Patrones positivos:")
        for pattern in pattern_analysis['good_patterns']:
            print(f"      - {pattern}")
    
    if pattern_analysis['anti_patterns']:
        print(f"  ❌ Anti-patrones detectados:")
        for anti_pattern in pattern_analysis['anti_patterns']:
            print(f"      - {anti_pattern}")
    
    if pattern_analysis['suggestions']:
        print(f"  💡 Sugerencias de mejora:")
        for suggestion in pattern_analysis['suggestions']:
            print(f"      - {suggestion}")
    
    # 3. Preparar datos para pipeline
    analysis_data = {
        "code": sample_legacy_code,
        "metrics": complexity_analysis,
        "patterns": pattern_analysis
    }
    
    print(f"\n✅ Análisis completo - Datos preparados para refactoring")
    return analysis_data

# Ejecutar análisis
legacy_analysis = await analyze_legacy_code()

🔍 Iniciando análisis automático del código legacy...

📊 ANÁLISIS DE COMPLEJIDAD:
  • Complejidad máxima: 8
  • Complejidad promedio: 8.0
  • Índice mantenibilidad: 80.9
  • Líneas de código: 31
  • Funciones complejas: ['process_user_data']
  ⚠️  PROBLEMAS DETECTADOS:
      - Función 'process_user_data' muy compleja (8 puntos)
      - TODOs pendientes en el código
      - Uso excesivo de print statements

🔍 ANÁLISIS DE PATRONES:
  ✅ Patrones positivos:
      - Funciones definidas (modularidad)
  ❌ Anti-patrones detectados:
      - Uso excesivo de print (usar logging)
      - TODOs sin resolver
      - Anidamiento profundo (7 niveles)
  💡 Sugerencias de mejora:
      - Resolver TODOs pendientes
      - Refactorizar para reducir complejidad
      - Reemplazar prints con logging estructurado
      - Extraer funciones para reducir anidamiento

✅ Análisis completo - Datos preparados para refactoring


In [53]:
# EJECUCIÓN DEL REFACTORING INTELIGENTE
async def execute_intelligent_refactoring():
    """Ejecutar el pipeline completo de refactoring"""
    print("🚀 Ejecutando pipeline de refactoring inteligente...")
    

    # Ejecutar pipeline con análisis previo
    refactoring_result = await refactoring_system.ainvoke(legacy_analysis)
    
    # Mostrar análisis técnico
    analysis = refactoring_result['original']['analysis']
    print(f"\n📋 ANÁLISIS TÉCNICO:")
    print(f"  • Prioridad de refactoring: {analysis.refactoring_priority}")
    print(f"  • Esfuerzo estimado: {analysis.estimated_effort}")
    print(f"  • Puntuación mantenibilidad: {analysis.maintainability_score}/10")
    
    print(f"\n⚠️  PROBLEMAS IDENTIFICADOS:")
    for issue in analysis.code_issues:
        print(f"    - {issue}")
    
    print(f"\n💡 SUGERENCIAS DE MEJORA:")
    for suggestion in analysis.improvement_suggestions:
        print(f"    - {suggestion}")
    
    # Mostrar código refactorizado
    refactored = refactoring_result['refactored']
    print(f"\n✨ CÓDIGO REFACTORIZADO:")
    print("="*60)
    print(refactored.improved_code)
    print("="*60)
    
    print(f"\n📝 RESUMEN DE CAMBIOS:")
    for change in refactored.changes_summary:
        print(f"    ✓ {change}")
    
    if refactored.performance_improvements:
        print(f"\n⚡ MEJORAS DE RENDIMIENTO:")
        for improvement in refactored.performance_improvements:
            print(f"    ⚡ {improvement}")
    
    if refactored.code_style_fixes:
        print(f"\n🎨 CORRECCIONES DE ESTILO:")
        for fix in refactored.code_style_fixes:
            print(f"    🎨 {fix}")
    
    return refactoring_result
    

# Ejecutar refactoring
print("Iniciando proceso de refactoring...")
refactoring_results = None


Iniciando proceso de refactoring...


In [54]:
refactoring_results = await execute_intelligent_refactoring()

🚀 Ejecutando pipeline de refactoring inteligente...

📋 ANÁLISIS TÉCNICO:
  • Prioridad de refactoring: Alta
  • Esfuerzo estimado: Medio
  • Puntuación mantenibilidad: 7/10

⚠️  PROBLEMAS IDENTIFICADOS:
    - Condicionales anidadas profundas en la lógica de descuentos (hasta 7 niveles)
    - TODO de validación de entrada sin resolver (riesgo de KeyError/TypeError)
    - Uso de print para registro de eventos (acopla E/S con lógica, dificulta observabilidad)
    - Fuga potencial de PII en logs (name/email)
    - Import faltante de 'time' (NameError en tiempo de ejecución)
    - Falta de tipado y docstring (disminuye legibilidad y herramientas estáticas)
    - Valores mágicos y cadenas hardcodeadas (umbrales, regiones, estados)
    - Uso de time.time() sin zona horaria (marca de tiempo no trazable/ambigua)
    - Función con múltiples responsabilidades (procesa datos y reporta progreso)
    - Duplicación de asignaciones en el dict 'processed' (verbosidad y riesgo de errores)
    - Sin mane